## **A Quantum Enhanced Markov Chain Monte Carlo  Algorithm : Experiments & Simulation** 

Our aim in this project is to explore the idea of an **Quantum Enhanced** MCMC algorithm, which was proposed initially by *David Layden et al* in the this [article](https://arxiv.org/abs/2203.12497). Markov Chain Monte-Carlo is one of the fundamental tools available for a sampling tasks that arises in wide variety of computational problems. One specific area of interest is sampling from thermal distribution of Spin-Glass type hamiltonians at low tempeatures, where the energy landscape turns into a space of sparse energy minimas separated by wide barren plateaus making it difficult for usual algorithms to sample effectively.

$ \mathbf{E}(s) = \sum_{ij} s_i J_{ij} s_j + \sum_{j} h_j s_j $

The usual approach to this task is through the **Metropolis Algorithm**, whihc essentially consisits of two steps :
1. **Proposal Step :**  We propose a transition from the current state *s* to another state *s'*, with some probability $\mathbf{Q}(s \rightarrow s')$
2. **Acceptance Step :** Based on the sampling inverse-temperature $\beta$, we accept or reject the proposed transition with the probability $\mathbf{T}(s \rightarrow s') = min(1, e^{-\beta (\mathbf{E}(s') - \mathbf{E}(s))})$

The expectation is that with sufficient iteration of the above algorithm i.e an MCMC chain of the accepted states ($ s_0 \rightarrow s_1 \rightarrow s_2 \rightarrow s_3$ . .. ) we can closely approximate the desired probability distribution $\mathbf{P}(s) = \frac{e^{-\beta \mathbf{E}(s)}}{\mathcal{Z}_{\beta}}$. However one significant caveat in this approach is that under a general setting we lack a good *'proposal mechanism'* that can effciently propose transitions which would be accepted with high enough probability and this considerably slows down the convergence of the MCMC chain to the desired distribution. Basic classical approcahes like the *uniform-sampling* where we pick randomly pick a state at uniform or *local-sampling* where we propose local changes often fail to sample the space effectively because either the states proposed have high energy difference or are too localised to the intial state. 

The aim with the *Quantum Enhanced MCMC* is to somehow enhance the proposal step such that it provides us with transitions that are more likely to be accepted, i.e the energy differnece between the initial and the proposed state is minimal. In the figure below, the essential difference in the approach to the proposition step of this algorithm has been highlighted.                                      

![proposition-strategy](DATA/figures/prop-strategy.jpeg)

###### Figures taken from https://arxiv.org/abs/2203.12497


Note how the proposed states are from is from a different energy minima and yet the energy difference between *s* and *s'* is low, compared to the *local* and *uniform* proposals it has both higher probability of being accepted and it enact sampling in a more global sense that is sees the entire energy landscape. Physically it could be thought analogous to *'quantum-mechanical-tunneliing'*, where particles can tunnel through narrow regions of high-energy barriers. In the next section we highlight the basic step of this enhanced proposition algorithm (refer as quMCMC)                                       


However, there exists sophisticated algoriths for special cases of the ising models that would be sampled dependeing upon their properties like connectivity, for example see Cluster-flipping algorithm, Swesden-Wang algorithm etc. but we will be excluding them for your discussions


#### **quMCMC : Algorithm**

The approach to built a quantum algorithm that can help us with better propositions can be designed based on your intuition of the available QAOA type algorithms where we stack layers of alternating operators applying the *Phase* and the *Mixer* Unitaries. There the role of the phase unitaries $\hat{U_p}$ is to apply phases corresponding the problem hamiltonian $\hat{H_p}$ and mixer unitary $\hat{U_m}$ acts to mix-up the phases of different basis states such as to allow transitions between them, the usual choice for being $ \hat{H}_m = \sum_i \hat{X}_i$ since it easier to implemnt on device and also does not commute with the $\hat{H}_p$. though for this algorithm we won't need to optimse the angles corresponding to this unitaries like the QAOA we use the same essential ingredeints.

Based on this ideas we construct an evolution hamiltonian of the form 
$$                      \hat{H}_{ev} = \gamma \hat{H}_p + (1- \gamma) \hat{H}_m
$$

Here $\gamma$ intorduced a relative weight between the two hamiltonians. The basic idea to enhancing the proposal mechanism here is to, start with a random state *s* and evolve it under the $\hat{U}_{ev} = e^{-i \tau \hat{H}_{ev}}$ for a short period of time and then measure it in a computational basis to get the proposed state *s'*, which is then processed into the *Acceptance step*.


#### Experiments on Statevector Simulators

In this section we will present the resukts we obtained from our experiments on the implementation of the quMCMC algorithm on statevector simulators. The implementation was done in `qiskit` as well as in `qulacs` backends both of which provides efficient statevector simulators that could further enhanced from CUDA based GPU-Accelarators. However, for our purpose it seemed fit to use the `qulacs` backend as it provided a considerable speed-up for the particular range of qubits we experimented with. 

###### Refer to: [Qulacs-Doc](http://docs.qulacs.org/en/latest/intro/0_about.html), [Qiskit-Doc](https://qiskit.org/documentation/tutorials/simulators/1_aer_provider.html)

Our implementation follows the exact steps mentioned in the algorithm. Paramters like relative weigtht $\gamma$, evolution time $\tau$ and the level of trotterization are picked randomly on every single circuit executions, however there ranges  

#### Experiment on Real Quantum Hardware

Next we illustrate the results we obtained from running our experiments on a Real Quantum Hardware. F
As indicated above, the margin of quantum advantage becomes prominent only when we move to higher number of qubits. 

__ Picture of IBM_guadalupe

We were given access to the device ibmq-guadalupe (14 qubits) with max. 300 circuits to run and 100000 shots per job. Although, we intended to explore a 2D-Ising model with large number of qubits, based on the given specifications we had to restrict our model accordingly. 

Based on the offline-online method used in https://arxiv.org/abs/2203.12497, we set up the proposal matrix first by running large number of circuits with various values of $\gamma$,t ($\gamma \in (0.26,0.6), t \in (1.6,9.6)$) and SPAM twirlings, and averaging over the whole data. Since we could only measure $10^5$ shots, number of spins had to be limited to 8 so that we could sample each transition atleast once (scales as $2^n \times 2^n$). 

Next we choose a model that respects the device connectivity, so that we dont require SWAP gates. The qubits 0,1,2,3,4,6,7,10 were chosen based on their lower CNOT & measurement error rates. The qubit to device mapping is shown below

__picture of chosen qubits

The Ising model was chosen as follows(shown as a graph). The coupling and local magnetic fields were chosen randomly, while respecting connectivity.

![graph-ising](DATA/figures/Ising_graph.png)

__model more info

Using the device data we get the following proposal matrix (log scale). We attempted SPAM twirling as mentioned in  https://arxiv.org/abs/2203.12497, but were unable to execute the two-qubit twirling properly. This probably leads to the assymmetry in the proposal matrix that we see here.

__prposal matrix

For a given matrix $M = \{m_{jk}\}$ to quantify the assymmetry we use the following measure,
$$
\chi^2 = \sum_{j<k} \frac{(m_{jk} - m_{kj})^2}{m_{jk} + m_{kj}} = 7255.5
$$


Based on our proposal matrix we run the usual classical sampling using M-H algorithm, only using the quantum data for proposal probabilities. We can compare the quantum strategy with classical methods by looking at acceptance probabilities and KL convergence towards exact solution

__Sampling probs

__KL convergence data 

#### Measuring magnetization

We can compare effectiveness of our method by looking at the magnetization values from classical and quantum methods wrt exact solution.

__magnetization at diff temps